# Hyperparameter tuning

This notebook focuses on tuning the hyperparameters of an initial network to find a better model on the training set. 
The starting point is a 4 layered feed forward neural network with 120, 60, 25 and 10 neurons per layer.

## Random search
To reduce computation a random search is preferred to a complete grid search.
A total of 100 models are tested.

In [1]:
import sys
sys.path.append("..")
from src.model import NeuralNetwork
import numpy as np
np.random.seed(1)

In [2]:
k = 2
neurons = [(120, 60 + i*k, 25 + j*k, 10) for i in (-2,-1,0,1,2) for j in (-2,-1,0,1,2)]

learning_rate = [0.001, 0.01, 0.1, 0.5]
momentum = [0.0, 0.01, 0.1, 1]
epochs = [100]
batch_size = [32]

param_dist = dict(neurons=neurons, 
                  learning_rate=learning_rate,
                  momentum=momentum, 
                  epochs=epochs, 
                  batch_size=batch_size)

results = NeuralNetwork.optimize_model(method="random", 
                                       param_grid=param_dist,
                                       dataset_path="../data/processed/extended/train_pca.csv", 
                                       iterations=100)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [3]:
print("Best: %f using %s\n" % (results.best_score_, results.best_params_))
means = results.cv_results_['mean_test_score']
stds = results.cv_results_['std_test_score']
params = results.cv_results_['params']
for mean, stdev, param in sorted(zip(means, stds, params), key=lambda x : -x[0])[:3]:
    print("%f (%f) with: %r" % (mean, stdev, param))
print("...")

Best: 0.629676 using {'neurons': (120, 62, 27, 10), 'momentum': 0.0, 'learning_rate': 0.1, 'epochs': 100, 'batch_size': 32}

0.629676 (0.053022) with: {'neurons': (120, 62, 27, 10), 'momentum': 0.0, 'learning_rate': 0.1, 'epochs': 100, 'batch_size': 32}
0.629015 (0.042463) with: {'neurons': (120, 64, 29, 10), 'momentum': 0.0, 'learning_rate': 0.1, 'epochs': 100, 'batch_size': 32}
0.627230 (0.052611) with: {'neurons': (120, 62, 27, 10), 'momentum': 0.1, 'learning_rate': 0.1, 'epochs': 100, 'batch_size': 32}
...


In [4]:

best_param = results.best_params_
best_param

{'neurons': (120, 62, 27, 10),
 'momentum': 0.0,
 'learning_rate': 0.1,
 'epochs': 100,
 'batch_size': 32}

# Test sets

The final model is evaluated on the test set to determine performances.

In [22]:
p = results.best_params_

model = NeuralNetwork.create_model(neurons=p["neurons"],
                                   learning_rate=p["learning_rate"], 
                                   momentum=0.0)

from src.data import Dataset
from sklearn.utils import class_weight

d = Dataset(dataset_path="../data/processed/extended/train_pca.csv", 
            test_size=0)

from keras.callbacks import EarlyStopping
stopper = EarlyStopping(monitor='accuracy', patience=3, verbose=1)
fit_params = dict(callbacks=[stopper])

x, y = d.get_splits()
class_weights = class_weight.compute_class_weight('balanced', np.unique(y), y)
weights_dict = dict(zip(np.unique(y), class_weights))

model.fit(x, y, class_weight=weights_dict, epochs=p["epochs"], batch_size=p["batch_size"], verbose=0, **fit_params)

Epoch 00016: early stopping


In [23]:
from src.utils import show_accuracy_loss

accuracy, loss = show_accuracy_loss(model, scaling="pca", test_dataset_path="../data/processed/extended")

7/7 [==============================] - 0s 2ms/step - loss: 1.9732 - accuracy: 0.6774

Accuracy:
	Mean: 0.6724625587463379 
	Standard deviation: 0.0403987395242968

Loss:
	Mean: 2.1437448978424074 
	Standard deviation: 0.21210886804849724
